# Data Preprocessing for Banking Customer Questions Classification

## 1. DATA LOADING & INITIAL SETUP

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from collections import Counter
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("1. DATA LOADING & INITIAL SETUP")
print("-" * 30)

# Load the raw data
print("Loading raw data...")
df = pd.read_excel('../data/raw/Коментари за сортиране.xlsx', header=None, skiprows=1)
df.columns = ['index', 'message_uid', 'created_dttm', 'question', 'department', 'col6', 'col7', 'market_segment']

print(f"   Raw data loaded: {df.shape}")
print(f"   Questions: {len(df)}")
print(f"   Unique departments: {df['department'].nunique()}")

# Display current class distribution
print(f"\nCurrent class distribution:")
class_dist = df['department'].value_counts()
for i, (dept, count) in enumerate(class_dist.items(), 1):
    percentage = (count / len(df)) * 100
    print(f"{i:2d}. {dept[:40]:<40} {count:4d} ({percentage:5.1f}%)")

1. DATA LOADING & INITIAL SETUP
------------------------------
Loading raw data...
   Raw data loaded: (1962, 8)
   Questions: 1962
   Unique departments: 15

Current class distribution:
 1. Пазар Ежедневно банкиране                 576 ( 29.4%)
 2. Други                                     318 ( 16.2%)
 3. Пазар Жилищни и ипотечни кредити          269 ( 13.7%)
 4. Пазар Потребителско кредитиране           144 (  7.3%)
 5. Пазар Разсрочени плащания                 120 (  6.1%)
 6. Пазар Малък бизнес                         95 (  4.8%)
 7. ДИРЕКЦИЯ БАНКОВИ ОПЕРАЦИИ                  83 (  4.2%)
 8. Пазар Спестяване и инвестиции              83 (  4.2%)
 9. Няколко пазара                             70 (  3.6%)
10. Пазар Банково застраховане                 66 (  3.4%)
11. ДИРЕКЦИЯ ПАЗАР АТМ И КАСОВА ДЕЙНОСТ        61 (  3.1%)
12. ПРАВНО                                     34 (  1.7%)
13. ДИРЕКЦИЯ КРЕДИТЕН РИСК ИНДИВИДУАЛНИ КЛИЕ   33 (  1.7%)
14. CRM                                       

## 2. CLASS CONSOLIDATION STRATEGY

In [2]:
print("Problem Analysis:")
print("   ДСК Лизинг: 1 sample (impossible to train/test)")
print("   CRM: 9 samples (high overfitting risk)")
print("   ПРАВНО: 34 samples (borderline for robust training)")
print("   Extreme imbalance ratio: 576:1")

print(f"\nSolution: Consolidate Support Services")
print("   Combining low-volume support departments into 'Support_Services'")

# Define class mapping strategy
CLASS_MAPPING = {
    # Support services consolidation (business logic: non-customer-facing support)
    'ПРАВНО': 'Support_Services',           # Legal support: 34 samples
    'CRM': 'Support_Services',              # Customer relationship: 9 samples  
    'ДСК Лизинг': 'Support_Services',       # Leasing services: 1 sample
    
    # Keep all other departments as-is (sufficient sample sizes)
    'Пазар Ежедневно банкиране': 'Пазар Ежедневно банкиране',
    'Други': 'Други',
    'Пазар Жилищни и ипотечни кредити': 'Пазар Жилищни и ипотечни кредити',
    'Пазар Потребителско кредитиране': 'Пазар Потребителско кредитиране',
    'Пазар Разсрочени плащания': 'Пазар Разсрочени плащания',
    'Пазар Малък бизнес': 'Пазар Малък бизнес',
    'ДИРЕКЦИЯ БАНКОВИ ОПЕРАЦИИ': 'ДИРЕКЦИЯ БАНКОВИ ОПЕРАЦИИ',
    'Пазар Спестяване и инвестиции': 'Пазар Спестяване и инвестиции',
    'Няколко пазара': 'Няколко пазара',
    'Пазар Банково застраховане': 'Пазар Банково застраховане',
    'ДИРЕКЦИЯ ПАЗАР АТМ И КАСОВА ДЕЙНОСТ': 'ДИРЕКЦИЯ ПАЗАР АТМ И КАСОВА ДЕЙНОСТ',
    'ДИРЕКЦИЯ КРЕДИТЕН РИСК ИНДИВИДУАЛНИ КЛИЕНТИ': 'ДИРЕКЦИЯ КРЕДИТЕН РИСК ИНДИВИДУАЛНИ КЛИЕНТИ'
}

# Apply class mapping
df['department_consolidated'] = df['department'].map(CLASS_MAPPING)

# Verify consolidation
print(f"\nAfter consolidation:")
consolidated_dist = df['department_consolidated'].value_counts()
for i, (dept, count) in enumerate(consolidated_dist.items(), 1):
    percentage = (count / len(df)) * 100
    print(f"{i:2d}. {dept[:40]:<40} {count:4d} ({percentage:5.1f}%)")

print(f"\nClass consolidation summary:")
print(f"   Classes reduced: {len(class_dist)} → {len(consolidated_dist)}")
print(f"   Minimum class size: {consolidated_dist.min()} samples")
print(f"   New imbalance ratio: {consolidated_dist.max()}:{consolidated_dist.min()} (was 576:1)")
print(f"   Support_Services created from: ПРАВНО + CRM + ДСК Лизинг = 44 samples")

Problem Analysis:
   ДСК Лизинг: 1 sample (impossible to train/test)
   CRM: 9 samples (high overfitting risk)
   ПРАВНО: 34 samples (borderline for robust training)
   Extreme imbalance ratio: 576:1

Solution: Consolidate Support Services
   Combining low-volume support departments into 'Support_Services'

After consolidation:
 1. Пазар Ежедневно банкиране                 576 ( 29.4%)
 2. Други                                     318 ( 16.2%)
 3. Пазар Жилищни и ипотечни кредити          269 ( 13.7%)
 4. Пазар Потребителско кредитиране           144 (  7.3%)
 5. Пазар Разсрочени плащания                 120 (  6.1%)
 6. Пазар Малък бизнес                         95 (  4.8%)
 7. ДИРЕКЦИЯ БАНКОВИ ОПЕРАЦИИ                  83 (  4.2%)
 8. Пазар Спестяване и инвестиции              83 (  4.2%)
 9. Няколко пазара                             70 (  3.6%)
10. Пазар Банково застраховане                 66 (  3.4%)
11. ДИРЕКЦИЯ ПАЗАР АТМ И КАСОВА ДЕЙНОСТ        61 (  3.1%)
12. Support_Services 

## 3. TEXT QUALITY ANALYSIS & CLEANING

In [3]:
# Very short questions
short_questions = df[df['question'].str.len() < 10]
print(f"   Very short questions (<10 chars): {len(short_questions)}")
if len(short_questions) > 0:
    print("     Examples:")
    for i, row in short_questions.head(5).iterrows():
        print(f"       '{row['question']}' → {row['department_consolidated']}")

# Very long questions (potential outliers)
long_questions = df[df['question'].str.len() > 500]
print(f"   Very long questions (>500 chars): {len(long_questions)}")

# Missing or null questions
null_questions = df['question'].isnull().sum()
print(f"   Null/missing questions: {null_questions}")

# Character encoding issues (non-Bulgarian characters)
bulgarian_pattern = re.compile(r'^[а-яА-ЯёЁ\s\d.,!?;:()"\'-]+$')
non_bulgarian = df[~df['question'].str.match(bulgarian_pattern, na=False)]
print(f"   Questions with non-Bulgarian chars: {len(non_bulgarian)}")

   Very short questions (<10 chars): 29
     Examples:
       'в bankway' → Други
       'да' → Други
       'трезор' → Пазар Ежедневно банкиране
       'ДОбър ден' → Други
       'Драстиии' → Други
   Very long questions (>500 chars): 17
   Null/missing questions: 0
   Questions with non-Bulgarian chars: 297


## 4.TEXT CLEANING & FILTERING

In [4]:
print(f"\nText cleaning strategy:")

# Define text cleaning function
def clean_bulgarian_text(text):
    """Clean Bulgarian text for NLP processing"""
    if pd.isna(text):
        return ""
    
    # Convert to string and lowercase
    text = str(text).lower()
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove leading/trailing whitespace
    text = text.strip()
    
    # Remove excessive punctuation (keep single punctuation)
    text = re.sub(r'[.]{2,}', '.', text)
    text = re.sub(r'[!]{2,}', '!', text)
    text = re.sub(r'[?]{2,}', '?', text)
    
    # Normalize quotes
    text = re.sub(r'[""„"]', '"', text)
    text = re.sub(r'[''`]', "'", text)
    
    return text

# Apply text cleaning
print("   Applying Bulgarian text normalization...")
df['question_cleaned'] = df['question'].apply(clean_bulgarian_text)

# Handle very short questions
print("   Handling very short questions...")
df['is_short_question'] = df['question_cleaned'].str.len() < 10
print(f"     Flagged {df['is_short_question'].sum()} short questions for review")

# Quality check after cleaning
cleaned_lengths = df['question_cleaned'].str.len()
print(f"\nText statistics after cleaning:")
print(f"   Average length: {cleaned_lengths.mean():.1f} characters")
print(f"   Median length: {cleaned_lengths.median():.1f} characters")
print(f"   Min length: {cleaned_lengths.min()} characters")
print(f"   Max length: {cleaned_lengths.max()} characters")
print(f"   Questions <10 chars: {(cleaned_lengths < 10).sum()}")

# Check for any remaining issues
print("Final quality assessment:")

# Duplicate questions after cleaning
duplicate_cleaned = df.duplicated(subset=['question_cleaned'], keep=False).sum()
print(f"   Duplicate questions (after cleaning): {duplicate_cleaned}")

if duplicate_cleaned > 0:
    duplicates_df = df[df.duplicated(subset=['question_cleaned'], keep=False)].sort_values('question_cleaned')
    if len(duplicates_df) > 0:
        for i, (idx, row) in enumerate(duplicates_df.head(4).iterrows()):
            print(f"       Raw: '{row['question'][:50]}...'")
            print(f"       Cleaned: '{row['question_cleaned'][:50]}...'")
            print(f"       Department: {row['department_consolidated']}")
            if i < len(duplicates_df) - 1:
                print(f"       ---")
    
    # Decision: Remove duplicates for cleaner training
    print(f"     Action: Removing duplicates, keeping first occurrence")
    df = df.drop_duplicates(subset=['question_cleaned'], keep='first')
    print(f"     Dataset size after deduplication: {len(df)} questions")

# Empty questions after cleaning
empty_cleaned = (df['question_cleaned'].str.len() == 0).sum()
print(f"   Empty questions (after cleaning): {empty_cleaned}")

# Class distribution validation
min_class_size = consolidated_dist.min()
print(f"   Minimum class size: {min_class_size} samples")
print(f"   All classes trainable: {'Yes' if min_class_size >= 5 else 'No'}")

print(f"\nData quality summary:")
print(f"   Clean questions ready: {len(df) - empty_cleaned}")
print(f"   Consolidated classes: {len(consolidated_dist)}")
print(f"   Ready for train/test split: {'Yes' if min_class_size >= 10 else 'Risky'}")


Text cleaning strategy:
   Applying Bulgarian text normalization...
   Handling very short questions...
     Flagged 29 short questions for review

Text statistics after cleaning:
   Average length: 65.7 characters
   Median length: 50.0 characters
   Min length: 2 characters
   Max length: 4094 characters
   Questions <10 chars: 29
Final quality assessment:
   Duplicate questions (after cleaning): 10
       Raw: 'Когато има технически проблем с АТМ, какви такси с...'
       Cleaned: 'когато има технически проблем с атм, какви такси с...'
       Department: ДИРЕКЦИЯ БАНКОВИ ОПЕРАЦИИ
       ---
       Raw: 'Когато има технически проблем с АТМ, какви такси с...'
       Cleaned: 'когато има технически проблем с атм, какви такси с...'
       Department: Пазар Ежедневно банкиране
       ---
       Raw: 'колко дебитни карти може да има един клиент...'
       Cleaned: 'колко дебитни карти може да има един клиент...'
       Department: Пазар Ежедневно банкиране
       ---
       Raw: 'колко д

## 5. TRAIN/TEST SPLIT

In [5]:
# Prepare data for splitting
print("Preparing for train/test split...")

# Remove empty questions if any
clean_df = df[df['question_cleaned'].str.len() > 0].copy()
print(f"   Removed {len(df) - len(clean_df)} empty questions")
print(f"   Final dataset size: {len(clean_df)} questions")

# Features and target for modeling
X = clean_df['question_cleaned']        # Primary feature for classification
y = clean_df['department_consolidated'] # Target variable

print(f"\nModeling dataset prepared:")
print(f"   Primary feature: question_cleaned (text)")
print(f"   Target: department_consolidated")
print(f"   Additional metadata will be preserved for analysis")

print(f"\nClass distribution for splitting:")
final_class_dist = y.value_counts()
for dept, count in final_class_dist.items():
    percentage = (count / len(clean_df)) * 100
    print(f"   • {dept[:35]:<35} {count:4d} ({percentage:5.1f}%)")

# Check if stratified split is possible
min_samples_per_class = final_class_dist.min()
test_size = 0.2
min_test_samples = int(min_samples_per_class * test_size)

print(f"\nSplit feasibility analysis:")
print(f"   Test size: {test_size} ({test_size*100}%)")
print(f"   Minimum samples per class: {min_samples_per_class}")
print(f"   Minimum test samples per class: {min_test_samples}")
print(f"   Stratified split possible: {'Yes' if min_test_samples >= 1 else 'No'}")

# Perform stratified split
try:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=test_size,
        random_state=42,
        stratify=y
    )
    
    print(f"\nStratified split successful!")
    print(f"   • Training set: {len(X_train)} questions")
    print(f"   • Test set: {len(X_test)} questions")
    
    # Verify stratification
    print(f"\nTrain/test distribution verification:")
    train_dist = y_train.value_counts().sort_index()
    test_dist = y_test.value_counts().sort_index()
    
    print(f"{'Class':<35} {'Train':<8} {'Test':<8} {'Train%':<8} {'Test%'}")
    print("-" * 70)
    for class_name in train_dist.index:
        train_count = train_dist[class_name]
        test_count = test_dist[class_name] if class_name in test_dist.index else 0
        train_pct = (train_count / len(X_train)) * 100
        test_pct = (test_count / len(X_test)) * 100 if test_count > 0 else 0
        print(f"{class_name[:34]:<35} {train_count:<8} {test_count:<8} {train_pct:<8.1f} {test_pct:<8.1f}")
    
except ValueError as e:
    print(f"Stratified split failed: {e}")
    print("   Falling back to random split...")
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=test_size,
        random_state=42
    )
    print(f"   • Training set: {len(X_train)} questions")
    print(f"   • Test set: {len(X_test)} questions")

Preparing for train/test split...
   Removed 0 empty questions
   Final dataset size: 1957 questions

Modeling dataset prepared:
   Primary feature: question_cleaned (text)
   Target: department_consolidated
   Additional metadata will be preserved for analysis

Class distribution for splitting:
   • Пазар Ежедневно банкиране            572 ( 29.2%)
   • Други                                317 ( 16.2%)
   • Пазар Жилищни и ипотечни кредити     269 ( 13.7%)
   • Пазар Потребителско кредитиране      144 (  7.4%)
   • Пазар Разсрочени плащания            120 (  6.1%)
   • Пазар Малък бизнес                    95 (  4.9%)
   • ДИРЕКЦИЯ БАНКОВИ ОПЕРАЦИИ             83 (  4.2%)
   • Пазар Спестяване и инвестиции         83 (  4.2%)
   • Няколко пазара                        70 (  3.6%)
   • Пазар Банково застраховане            66 (  3.4%)
   • ДИРЕКЦИЯ ПАЗАР АТМ И КАСОВА ДЕЙНОСТ   61 (  3.1%)
   • Support_Services                      44 (  2.2%)
   • ДИРЕКЦИЯ КРЕДИТЕН РИСК ИНДИВИДУАЛНИ   

## 6. DATA EXPORT & SUMMARY

In [6]:
print("Preparing features for model development...")

# Create feature engineering dataset
feature_df = clean_df.copy()

# Basic text features
feature_df['text_length'] = feature_df['question_cleaned'].str.len()
feature_df['word_count'] = feature_df['question_cleaned'].str.split().str.len()
feature_df['avg_word_length'] = feature_df['text_length'] / feature_df['word_count']

# Question type indicators
feature_df['has_question_mark'] = feature_df['question_cleaned'].str.contains(r'\?', na=False)  
feature_df['has_numbers'] = feature_df['question_cleaned'].str.contains(r'\d', na=False)
feature_df['has_uppercase'] = feature_df['question'].str.contains(r'[А-Я]', na=False)

print(f"Basic text features created:")
print(f"   Text length statistics: mean={feature_df['text_length'].mean():.1f}, std={feature_df['text_length'].std():.1f}")
print(f"   Word count statistics: mean={feature_df['word_count'].mean():.1f}, std={feature_df['word_count'].std():.1f}")
print(f"   Questions with '?': {feature_df['has_question_mark'].sum()} ({(feature_df['has_question_mark'].mean()*100):.1f}%)")
print(f"   Questions with numbers: {feature_df['has_numbers'].sum()} ({(feature_df['has_numbers'].mean()*100):.1f}%)")

# Prepare final datasets for export
print("Preparing datasets for export...")

# Training dataset - Modeling + Analysis
train_df = pd.DataFrame({
    # Core modeling columns
    'question_cleaned': X_train,
    'department_target': y_train,
    
    # Analysis and debugging columns
    'message_uid': clean_df.loc[X_train.index, 'message_uid'],
    'question_original': clean_df.loc[X_train.index, 'question'],
    'department_original': clean_df.loc[X_train.index, 'department'],
    'is_short_question': clean_df.loc[X_train.index, 'is_short_question']
})

# Test dataset - Modeling + Analysis  
test_df = pd.DataFrame({
    # Core modeling columns
    'question_cleaned': X_test,
    'department_target': y_test,
    
    # Analysis and debugging columns
    'message_uid': clean_df.loc[X_test.index, 'message_uid'],
    'question_original': clean_df.loc[X_test.index, 'question'],
    'department_original': clean_df.loc[X_test.index, 'department'],
    'is_short_question': clean_df.loc[X_test.index, 'is_short_question']
})

print(f"\nDataset structure:")
print(f"   Modeling columns: question_cleaned, department_target")
print(f"   Analysis columns: message_uid, question_original, department_original, is_short_question")
print(f"   Use modeling columns for training/testing")
print(f"   Use analysis columns for error analysis and business insights")

# Export to processed data folder
import os
os.makedirs('../data/processed', exist_ok=True)

train_df.to_csv('../data/processed/train_data.csv', index=False, encoding='utf-8')
test_df.to_csv('../data/processed/test_data.csv', index=False, encoding='utf-8')

# Also save class mapping for future reference
class_mapping_df = pd.DataFrame(list(CLASS_MAPPING.items()), columns=['original_department', 'consolidated_department'])
class_mapping_df.to_csv('../data/processed/class_mapping.csv', index=False, encoding='utf-8')

print(f"Data exported successfully:")
print(f"   Train dataset: ../data/processed/train_data.csv ({len(train_df)} rows)")
print(f"   Test dataset: ../data/processed/test_data.csv ({len(test_df)} rows)")
print(f"   Class mapping: ../data/processed/class_mapping.csv")
print(f"   Format: Modeling columns + Analysis metadata")
print(f"   Ready for both training and comprehensive evaluation")

# Final summary
print(f"\nPREPROCESSING SUMMARY:")
print(f"   Original dataset: {len(df)} questions, {df['department'].nunique()} classes")
print(f"   Final dataset: {len(clean_df)} questions, {len(consolidated_dist)} classes")
print(f"   Class consolidation: Support_Services = ПРАВНО + CRM + ДСК Лизинг")
print(f"   Text cleaning: Bulgarian normalization applied")
print(f"   Train/test split: {len(X_train)}/{len(X_test)} (stratified)")
print(f"   Minimum class size: {consolidated_dist.min()} samples")
print(f"   Imbalance ratio: {consolidated_dist.max()}:{consolidated_dist.min()}")

print(f"\nREADY FOR MODEL DEVELOPMENT:")
print(f"   Text classification: 13-class problem")
print(f"   Features: Clean Bulgarian text + basic text features")
print(f"   Target: Consolidated departments")
print(f"   Data quality: High (no duplicates, clean text)")
print(f"   Next step: 03_model_development.ipynb")

print(f"\nPreprocessing completed successfully!")

Preparing features for model development...
Basic text features created:
   Text length statistics: mean=65.8, std=123.2
   Word count statistics: mean=11.2, std=19.8
   Questions with '?': 403 (20.6%)
   Questions with numbers: 193 (9.9%)
Preparing datasets for export...

Dataset structure:
   Modeling columns: question_cleaned, department_target
   Analysis columns: message_uid, question_original, department_original, is_short_question
   Use modeling columns for training/testing
   Use analysis columns for error analysis and business insights
Data exported successfully:
   Train dataset: ../data/processed/train_data.csv (1565 rows)
   Test dataset: ../data/processed/test_data.csv (392 rows)
   Class mapping: ../data/processed/class_mapping.csv
   Format: Modeling columns + Analysis metadata
   Ready for both training and comprehensive evaluation

PREPROCESSING SUMMARY:
   Original dataset: 1957 questions, 15 classes
   Final dataset: 1957 questions, 13 classes
   Class consolidation